# GCP Play with LLMs

Ref: https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/text-bison?project=playpen-23f5behttps://console.cloud.google.com/vertex-ai/publishers/google/model-garden/text-bison?project=playpen-23f5be


In [1]:
import numpy as np
import pandas as pd
import json
import time

#from google.colab import auth as google_auth
#google_auth.authenticate_user()

import vertexai
#from vertexai.preview.language_models import TextGenerationModel, TextEmbeddingModel
from vertexai.language_models import TextGenerationModel, TextEmbeddingModel


## Project ID and Region

For multiple projects, do:
```
PROJECT_ID = "[your-project-id]"

# Set the project id
! gcloud config set project {PROJECT_ID}
```

In [4]:
PROJECT_ID = ! gcloud config get core/project
PROJECT_ID = PROJECT_ID[0]

REGION = "europe-west2"

PROJECT_ID, REGION

('playpen-321194', 'europe-west2')

## Text Generation

### Call from terminal using

In [5]:
%env PROJECT_ID=$PROJECT_ID

env: PROJECT_ID=playpen-321194


In [6]:
# The input prompt
! cat interview_prompt.json

{
    "instances": [
        { "prompt": "Give me ten interview questions for the role of program manager."}
    ], 
    "parameters": {"temperature": 0.2, "maxOutputTokens": 256, "topK": 40, "topP": 0.95 }
}



In [7]:
! curl -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @interview_prompt.json \
https://us-central1-aiplatform.googleapis.com/v1/projects/$PROJECT_ID/locations/us-central1/publishers/google/models/text-bison:predict


{
  "predictions": [
    {
      "safetyAttributes": {
        "blocked": false,
        "scores": [
          0.6,
          0.1
        ],
        "categories": [
          "Finance",
          "Health"
        ]
      },
      "citationMetadata": {
        "citations": []
      },
      "content": " 1. **What is your experience in managing programs?**\n2. **What are your strengths and weaknesses as a program manager?**\n3. **What is your approach to managing a program?**\n4. **How do you handle risks and challenges in a program?**\n5. **How do you communicate with stakeholders in a program?**\n6. **How do you measure success in a program?**\n7. **What is your experience in working with cross-functional teams?**\n8. **What is your experience in managing budgets and resources?**\n9. **What is your experience in managing change in a program?**\n10. **What are your career goals as a program manager?**"
    }
  ],
  "metadata": {
    "tokenMetadata": {
      "outputTokenCount": {
       

### Vertex AI SDK for Python

In [8]:

def interview(temperature: float = .2) -> None:
    """Ideation example with a Large Language Model"""

    # TODO developer - override these parameters as needed:
    parameters = {
        "temperature": temperature,
        "max_output_tokens": 256,   
        "top_p": .8,                
        "top_k": 40,                 
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        'Give me ten interview questions for the role of program manager.',
        **parameters,
    )
    print(f"Response from Model: \n{response.text}")
    

In [9]:
interview()

Response from Model: 
1. What is your experience with project management?
2. What is your process for managing a project?
3. How do you handle unexpected challenges or roadblocks?
4. How do you communicate with stakeholders?
5. How do you measure the success of a project?
6. What are your strengths and weaknesses as a project manager?
7. What are your salary expectations?
8. What are your career goals?
9. Why are you interested in this position?
10. What questions do you have for me?


### From Outside GCP such as Google Colab

In [9]:

vertexai.init(project=PROJECT_ID, location="us-central1")

def predict_large_language_model_sample(
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    content: str,
    tuned_model_name: str = "",
    ):
    """Predict using a Large Language Model."""
    
    model = (
        TextGenerationModel.from_pretrained(model_name) if not tuned_model_name 
        else model.get_tuned_model(tuned_model_name)
    )
    
    response = model.predict(
        content,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)
    
    print(f"Response from Model: \n{response.text}")
    
predict_large_language_model_sample("text-bison@001", 0.2, 256, 0.8, 40, 
                                    """Give me ten interview questions for the role of program manager""")



Response from Model: 
1. What is your experience with project management?
2. What is your process for managing a project?
3. How do you handle unexpected challenges or roadblocks?
4. How do you communicate with stakeholders?
5. How do you measure the success of a project?
6. What are your strengths and weaknesses as a project manager?
7. What are your salary expectations?
8. What are your career goals?
9. What are your thoughts on the company's culture?
10. Why are you interested in this position?


## Embeddings

### Terminal

In [10]:
# The input text
! cat embedding_request.json

{
  "instances": [
    { "content": "Roses are red, violets are blue; Sugar is sweet, and so are you."}
  ],
}


In [11]:
! curl -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json; charset=utf-8" \
-d @embedding_request.json \
"https://us-central1-aiplatform.googleapis.com/v1/projects/$PROJECT_ID/locations/us-central1/publishers/google/models/textembedding-gecko:predict"


{
  "predictions": [
    {
      "embeddings": {
        "statistics": {
          "truncated": false,
          "token_count": 19
        },
        "values": [
          -0.021640053018927574,
          -0.022342270240187645,
          0.0096256602555513382,
          0.00980695616453886,
          -0.0037967897951602936,
          -0.0022547577973455191,
          -0.012095913290977478,
          0.005097963847219944,
          0.0013537256745621562,
          -0.0056282766163349152,
          -0.023833643645048141,
          -0.03117951937019825,
          0.061525564640760422,
          -0.00894611980766058,
          0.023603029549121857,
          0.0013698431430384517,
          -0.012830539606511593,
          -0.048486240208148956,
          -0.0089709488674998283,
          0.00053539063083007932,
          -0.03362337127327919,
          -0.045788396149873734,
          -0.009663531556725502,
          0.034413404762744904,
          -0.042952544987201691,
          -0.1130

### Python SDK

In [12]:
#from vertexai.preview.language_models import TextEmbeddingModel
from vertexai.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko")

"""
get_embeddings() returns a list of embeddings that correspond to the list of input text.
Each embedding has 
    'values' : [float]
    'statistics': {'truncated': False, 'token_count': 5.0}}}
"""
embeddings = model.get_embeddings(["Dinner in New York City"])



In [26]:

for embedding in embeddings:
    vector = np.array(embedding.values)
    print(vector.shape)
    print(vector[:10])
    print(embedding.statistics)

(768,)
[-0.02773579  0.00554794  0.01364524  0.01273699  0.01570771 -0.04658821
 -0.00406408  0.03175907  0.0217345  -0.01409258]
TextEmbeddingStatistics(token_count=5.0, truncated=False)


## Semantic Search using Embeddings

In [37]:
model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

In [38]:
records_df = pd.read_json("../data/wide_and_deep_trainer_container_tests_input.jsonl",  lines=True)
records_df.head()

,textContent,classificationAnnotation,dataItemResourceLabels
0,"Cats are good pets, for they are clean and are...","{'displayName': 'FirstClass', 'annotationResou...",{'aiplatform.googleapis.com/ml_use': 'training'}
1,More RVs were seen in the storage lot than at ...,"{'displayName': 'SecondClass', 'annotationReso...",{'aiplatform.googleapis.com/ml_use': 'training'}
2,"When he asked her favorite number, she answere...","{'displayName': 'FirstClass', 'annotationResou...",{'aiplatform.googleapis.com/ml_use': 'training'}
3,Greetings from the real universe.,"{'displayName': 'SecondClass', 'annotationReso...",{'aiplatform.googleapis.com/ml_use': 'training'}
4,As he entered the church he could hear the sof...,"{'displayName': 'FirstClass', 'annotationResou...",{'aiplatform.googleapis.com/ml_use': 'training'}


In [42]:
def get_embedding(text):
    get_embedding.counter += 1
    try:
        if get_embedding.counter % 100 == 0:
            time.sleep(3)
        return model.get_embeddings([text])[0].values
    except:
        return []


get_embedding.counter = 0

# This may take several minutes to complete.
#records_df["embedding"] = records_df["textContent"].apply(lambda x: get_embedding(x))
records_df = records_df.assign(embedding = records_df["textContent"].apply(lambda x: get_embedding(x)))

In [43]:
records_df.head()

,textContent,classificationAnnotation,dataItemResourceLabels,embedding
0,"Cats are good pets, for they are clean and are...","{'displayName': 'FirstClass', 'annotationResou...",{'aiplatform.googleapis.com/ml_use': 'training'},"[0.037178341299295425, -0.010241042822599411, ..."
1,More RVs were seen in the storage lot than at ...,"{'displayName': 'SecondClass', 'annotationReso...",{'aiplatform.googleapis.com/ml_use': 'training'},"[-0.011439577676355839, -0.032596345990896225,..."
2,"When he asked her favorite number, she answere...","{'displayName': 'FirstClass', 'annotationResou...",{'aiplatform.googleapis.com/ml_use': 'training'},"[0.020134523510932922, -0.0022515689488500357,..."
3,Greetings from the real universe.,"{'displayName': 'SecondClass', 'annotationReso...",{'aiplatform.googleapis.com/ml_use': 'training'},"[-0.016334038227796555, 0.0007779737352393568,..."
4,As he entered the church he could hear the sof...,"{'displayName': 'FirstClass', 'annotationResou...",{'aiplatform.googleapis.com/ml_use': 'training'},"[0.005115862935781479, -0.034255024045705795, ..."


In [ ]:
# def embedding_chunks(text_chunks):
#     counter += 1
#     try:
#         if counter % 100 == 0:
#             time.sleep(3)
#         return model.get_embeddings([text])[0].values
#     except:
#         return []


#df.assign(emb=lambda _df: embedding_chunks(_df["textContent"]))

#df.head()

## Promp